In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import pickle
import pdb
import json
import numpy as np

from utils import *
from GCN.GCN import GCN
from GAT.GAT import GAT
from Dense.Dense import Dense
from full_data_process import graphDataProcess
from sub_data_process import subGraphProcess

from adjacency_functions import *
from feature_functions import *
from label_functions import *
from train_test_functions import *

In [3]:
def full_graph_process(param_dict, data_dir, full_processed_path, full_redo):
    if (not os.path.exists(full_processed_path)) or full_redo:
        raw_data_path = param_dict['gen_params']['raw_data_path']
        full_names_dict = param_dict['full_names_dict']
        full_redo_dict = param_dict['full_redo_dict']
        graph_data_obj = graphDataProcess(raw_data_path, data_dir, full_names_dict, full_redo_dict)
        graph_data_obj.run_all()
        pickle.dump(graph_data_obj, open(full_processed_path,'wb'))
    else:
        graph_data_obj = pickle.load(open(full_processed_path,'rb'))
    return graph_data_obj

def sub_graph_process(param_dict, data_path, full_processed_path, sub_processed_path, sub_redo):
    if (not os.path.exists(sub_processed_path)) or sub_redo:
        sampling_params = param_dict['sampling_params']
        sub_names_dict = param_dict['sub_names_dict']
        sub_redo_dict = param_dict['sub_redo_dict']
        sub_functions_dict = get_func_dict(param_dict['sub_functions_dict'])
        subgraph_data_obj = subGraphProcess(full_processed_path, data_path, sub_names_dict, sub_redo_dict, sub_functions_dict, sampling_params)
        subgraph_data_obj.run_all()
        pickle.dump(subgraph_data_obj, open(sub_processed_path,'wb'))
    else:
        subgraph_data_obj = pickle.load(open(sub_processed_path,'rb'))
    return subgraph_data_obj

In [4]:
ver = 'v0.1'
param_path = '/home/ds-team/aaron/other/MoonBoard/data/train_test/pytorch/graphNet/GraphNet/' + ver + '/params.json'
param_dict = json.load(open(param_path,'r'))

# Unwrap and set general parameters

In [5]:
model_type = param_dict['gen_params']['model_type']
ver = param_dict['gen_params']['ver']
data_dir = param_dict['gen_params']['data_dir']
result_dir = param_dict['gen_params']['result_dir']

data_path, result_path = set_paths(model_type, ver, data_dir, result_dir)

full_processed_name = param_dict['gen_params']['full_processed_name']
sub_processed_name = param_dict['gen_params']['sub_processed_name']

full_processed_path = data_dir+full_processed_name
sub_processed_path = data_path+sub_processed_name

full_redo = param_dict['gen_params']['full_redo']
sub_redo = param_dict['gen_params']['sub_redo']

In [6]:
graph_data_obj = full_graph_process(param_dict, data_dir, full_processed_path, full_redo)

In [7]:
subgraph_data_obj = sub_graph_process(param_dict, data_path, full_processed_path, sub_processed_path, sub_redo)

Sampling core nodes
Getting samples node features
Getting samples node adjacency
Getting samples node labels


In [8]:
split_ratio_dict = param_dict['split_ratio_dict']
target_grade = -1
features, adj, labels, idx_train, idx_dev, idx_test = sample_and_load_pytorch_data(subgraph_data_obj, split_ratio_dict, result_path, target_grade, sub_redo)

In [9]:
num_labels = len(list(set(list(np.asarray(labels)))))

In [10]:
dense_params = param_dict['dense_params']
if dense_params:
    num_epochs = dense_params['num_epochs']
    model = Dense(nfeat=features.shape[1],
                nhid_list=dense_params['hidden'],
                nclass=num_labels,
                dropout=dense_params['dropout'])

    optimizer = optim.Adam(model.parameters(),lr=dense_params['lr'], weight_decay=dense_params['weight_decay'])

In [11]:
gcn_params = param_dict['gcn_params']
if gcn_params:
    num_epochs = gcn_params['num_epochs']
    model = GCN(nfeat=features.shape[1],
                nhid_list=gcn_params['hidden'],
                nclass=num_labels,
                dropout=gcn_params['dropout'])

    optimizer = optim.Adam(model.parameters(),lr=gcn_params['lr'], weight_decay=gcn_params['weight_decay'])

In [12]:
gat_params = param_dict['gat_params']
if gat_params['on']:
    num_epochs = gat_params['num_epochs']
    model = GAT(nfeat=features.shape[1],
                nhid=gat_params['hidden'],
                nclass=num_labels,
                dropout=gat_params['dropout'],
                alpha=gat_params['alpha'],
                nheads=gat_params['nb_heads'])

    optimizer = optim.Adam(model.parameters(),lr=gat_params['lr'], weight_decay=gat_params['weight_decay'])

In [13]:
model

GCN(
  (gc_list): ModuleList(
    (0): GraphConvolution (11140 -> 128)
    (1): GraphConvolution (128 -> 12)
  )
)

In [14]:
train_dict = {}
train_dict['optimizer'] = optimizer
train_dict['features'] = features
train_dict['adj'] = adj
train_dict['labels'] = labels
train_dict['idx_train'] = idx_train
train_dict['idx_val'] = idx_dev
train_dict['num_epochs'] = num_epochs
model = run_train(model, train_dict)

Epoch: 0001 loss_train: 2.5236 acc_train: 0.0000 loss_val: 2.5045 acc_val: 0.0000 time: 0.4145s
Epoch: 0002 loss_train: 2.5196 acc_train: 0.0000 loss_val: 2.5033 acc_val: 0.0000 time: 0.4038s
Epoch: 0003 loss_train: 2.5157 acc_train: 0.0001 loss_val: 2.5022 acc_val: 0.0041 time: 0.3798s
Epoch: 0004 loss_train: 2.5120 acc_train: 0.0035 loss_val: 2.5012 acc_val: 0.0671 time: 0.3978s
Epoch: 0005 loss_train: 2.5082 acc_train: 0.0854 loss_val: 2.5001 acc_val: 0.0758 time: 0.3768s
Epoch: 0006 loss_train: 2.5047 acc_train: 0.0961 loss_val: 2.4991 acc_val: 0.0758 time: 0.3999s
Epoch: 0007 loss_train: 2.5006 acc_train: 0.0961 loss_val: 2.4981 acc_val: 0.0758 time: 0.3971s
Epoch: 0008 loss_train: 2.4969 acc_train: 0.0961 loss_val: 2.4969 acc_val: 0.0758 time: 0.4060s
Epoch: 0009 loss_train: 2.4933 acc_train: 0.0961 loss_val: 2.4958 acc_val: 0.0758 time: 0.4177s
Epoch: 0010 loss_train: 2.4893 acc_train: 0.0961 loss_val: 2.4946 acc_val: 0.0758 time: 0.3719s
Epoch: 0011 loss_train: 2.4850 acc_train

Epoch: 0087 loss_train: 2.3511 acc_train: 0.1212 loss_val: 2.3688 acc_val: 0.1009 time: 0.3799s
Epoch: 0088 loss_train: 2.3505 acc_train: 0.1196 loss_val: 2.3681 acc_val: 0.1009 time: 0.3939s
Epoch: 0089 loss_train: 2.3507 acc_train: 0.1177 loss_val: 2.3675 acc_val: 0.1019 time: 0.3945s
Epoch: 0090 loss_train: 2.3486 acc_train: 0.1306 loss_val: 2.3670 acc_val: 0.1019 time: 0.3850s
Epoch: 0091 loss_train: 2.3476 acc_train: 0.1374 loss_val: 2.3666 acc_val: 0.1019 time: 0.3756s
Epoch: 0092 loss_train: 2.3483 acc_train: 0.1250 loss_val: 2.3662 acc_val: 0.1019 time: 0.3633s
Epoch: 0093 loss_train: 2.3501 acc_train: 0.1180 loss_val: 2.3658 acc_val: 0.1029 time: 0.4034s
Epoch: 0094 loss_train: 2.3489 acc_train: 0.1244 loss_val: 2.3653 acc_val: 0.1019 time: 0.4212s
Epoch: 0095 loss_train: 2.3486 acc_train: 0.1165 loss_val: 2.3649 acc_val: 0.0993 time: 0.3662s
Epoch: 0096 loss_train: 2.3492 acc_train: 0.1218 loss_val: 2.3644 acc_val: 0.1039 time: 0.3674s
Epoch: 0097 loss_train: 2.3485 acc_train

Epoch: 0173 loss_train: 2.3295 acc_train: 0.1481 loss_val: 2.3362 acc_val: 0.1434 time: 0.4005s
Epoch: 0174 loss_train: 2.3315 acc_train: 0.1523 loss_val: 2.3360 acc_val: 0.1434 time: 0.3524s
Epoch: 0175 loss_train: 2.3339 acc_train: 0.1437 loss_val: 2.3358 acc_val: 0.1357 time: 0.3653s
Epoch: 0176 loss_train: 2.3290 acc_train: 0.1476 loss_val: 2.3354 acc_val: 0.1342 time: 0.3807s
Epoch: 0177 loss_train: 2.3296 acc_train: 0.1429 loss_val: 2.3351 acc_val: 0.1342 time: 0.4020s
Epoch: 0178 loss_train: 2.3324 acc_train: 0.1428 loss_val: 2.3348 acc_val: 0.1357 time: 0.4037s
Epoch: 0179 loss_train: 2.3299 acc_train: 0.1416 loss_val: 2.3345 acc_val: 0.1347 time: 0.3582s
Epoch: 0180 loss_train: 2.3332 acc_train: 0.1451 loss_val: 2.3343 acc_val: 0.1367 time: 0.3867s
Epoch: 0181 loss_train: 2.3310 acc_train: 0.1426 loss_val: 2.3342 acc_val: 0.1347 time: 0.4066s
Epoch: 0182 loss_train: 2.3306 acc_train: 0.1378 loss_val: 2.3341 acc_val: 0.1367 time: 0.3933s
Epoch: 0183 loss_train: 2.3325 acc_train

Epoch: 0259 loss_train: 2.3243 acc_train: 0.1413 loss_val: 2.3217 acc_val: 0.1500 time: 0.3677s
Epoch: 0260 loss_train: 2.3272 acc_train: 0.1363 loss_val: 2.3217 acc_val: 0.1536 time: 0.3826s
Epoch: 0261 loss_train: 2.3252 acc_train: 0.1532 loss_val: 2.3215 acc_val: 0.1536 time: 0.3592s
Epoch: 0262 loss_train: 2.3224 acc_train: 0.1473 loss_val: 2.3214 acc_val: 0.1521 time: 0.3461s
Epoch: 0263 loss_train: 2.3259 acc_train: 0.1340 loss_val: 2.3213 acc_val: 0.1526 time: 0.3572s
Epoch: 0264 loss_train: 2.3280 acc_train: 0.1387 loss_val: 2.3212 acc_val: 0.1572 time: 0.3883s
Epoch: 0265 loss_train: 2.3275 acc_train: 0.1431 loss_val: 2.3211 acc_val: 0.1572 time: 0.3663s
Epoch: 0266 loss_train: 2.3253 acc_train: 0.1425 loss_val: 2.3208 acc_val: 0.1567 time: 0.3621s
Epoch: 0267 loss_train: 2.3294 acc_train: 0.1410 loss_val: 2.3205 acc_val: 0.1572 time: 0.3649s
Epoch: 0268 loss_train: 2.3218 acc_train: 0.1450 loss_val: 2.3204 acc_val: 0.1582 time: 0.3784s
Epoch: 0269 loss_train: 2.3223 acc_train

Epoch: 0345 loss_train: 2.3165 acc_train: 0.1534 loss_val: 2.3175 acc_val: 0.1521 time: 0.3786s
Epoch: 0346 loss_train: 2.3241 acc_train: 0.1381 loss_val: 2.3174 acc_val: 0.1531 time: 0.3886s
Epoch: 0347 loss_train: 2.3231 acc_train: 0.1451 loss_val: 2.3171 acc_val: 0.1531 time: 0.4262s
Epoch: 0348 loss_train: 2.3172 acc_train: 0.1431 loss_val: 2.3167 acc_val: 0.1516 time: 0.4967s
Epoch: 0349 loss_train: 2.3200 acc_train: 0.1506 loss_val: 2.3162 acc_val: 0.1516 time: 0.5334s
Epoch: 0350 loss_train: 2.3186 acc_train: 0.1421 loss_val: 2.3157 acc_val: 0.1521 time: 0.5080s
Epoch: 0351 loss_train: 2.3189 acc_train: 0.1501 loss_val: 2.3152 acc_val: 0.1521 time: 0.5197s
Epoch: 0352 loss_train: 2.3246 acc_train: 0.1459 loss_val: 2.3150 acc_val: 0.1531 time: 0.4405s
Epoch: 0353 loss_train: 2.3186 acc_train: 0.1494 loss_val: 2.3147 acc_val: 0.1536 time: 0.4109s
Epoch: 0354 loss_train: 2.3228 acc_train: 0.1385 loss_val: 2.3146 acc_val: 0.1536 time: 0.3841s
Epoch: 0355 loss_train: 2.3212 acc_train

...

Epoch: 1635 loss_train: 2.3241 acc_train: 0.1446 loss_val: 2.3116 acc_val: 0.1521 time: 0.4483s
Epoch: 1636 loss_train: 2.3253 acc_train: 0.1357 loss_val: 2.3114 acc_val: 0.1510 time: 0.3728s
Epoch: 1637 loss_train: 2.3144 acc_train: 0.1559 loss_val: 2.3111 acc_val: 0.1510 time: 0.4010s
Epoch: 1638 loss_train: 2.3192 acc_train: 0.1432 loss_val: 2.3107 acc_val: 0.1516 time: 0.4249s
Epoch: 1639 loss_train: 2.3194 acc_train: 0.1448 loss_val: 2.3104 acc_val: 0.1521 time: 0.4042s
Epoch: 1640 loss_train: 2.3134 acc_train: 0.1363 loss_val: 2.3103 acc_val: 0.1516 time: 0.3899s
Epoch: 1641 loss_train: 2.3192 acc_train: 0.1509 loss_val: 2.3105 acc_val: 0.1516 time: 0.4029s
Epoch: 1642 loss_train: 2.3196 acc_train: 0.1446 loss_val: 2.3107 acc_val: 0.1521 time: 0.4010s
Epoch: 1643 loss_train: 2.3173 acc_train: 0.1404 loss_val: 2.3110 acc_val: 0.1521 time: 0.4160s
Epoch: 1644 loss_train: 2.3192 acc_train: 0.1476 loss_val: 2.3113 acc_val: 0.1510 time: 0.3982s
Epoch: 1645 loss_train: 2.3183 acc_train

Epoch: 1721 loss_train: 2.3179 acc_train: 0.1456 loss_val: 2.3116 acc_val: 0.1531 time: 0.3936s
Epoch: 1722 loss_train: 2.3169 acc_train: 0.1551 loss_val: 2.3113 acc_val: 0.1510 time: 0.3983s
Epoch: 1723 loss_train: 2.3249 acc_train: 0.1325 loss_val: 2.3111 acc_val: 0.1510 time: 0.3913s
Epoch: 1724 loss_train: 2.3216 acc_train: 0.1363 loss_val: 2.3110 acc_val: 0.1510 time: 0.3998s
Epoch: 1725 loss_train: 2.3239 acc_train: 0.1450 loss_val: 2.3111 acc_val: 0.1510 time: 0.3917s
Epoch: 1726 loss_train: 2.3205 acc_train: 0.1381 loss_val: 2.3114 acc_val: 0.1510 time: 0.3790s
Epoch: 1727 loss_train: 2.3186 acc_train: 0.1473 loss_val: 2.3113 acc_val: 0.1510 time: 0.3879s
Epoch: 1728 loss_train: 2.3197 acc_train: 0.1462 loss_val: 2.3113 acc_val: 0.1516 time: 0.3956s
Epoch: 1729 loss_train: 2.3155 acc_train: 0.1444 loss_val: 2.3114 acc_val: 0.1521 time: 0.3821s
Epoch: 1730 loss_train: 2.3221 acc_train: 0.1390 loss_val: 2.3117 acc_val: 0.1516 time: 0.3940s
Epoch: 1731 loss_train: 2.3173 acc_train

Epoch: 1807 loss_train: 2.3163 acc_train: 0.1498 loss_val: 2.3111 acc_val: 0.1526 time: 0.4082s
Epoch: 1808 loss_train: 2.3224 acc_train: 0.1416 loss_val: 2.3107 acc_val: 0.1536 time: 0.4210s
Epoch: 1809 loss_train: 2.3194 acc_train: 0.1478 loss_val: 2.3104 acc_val: 0.1526 time: 0.3841s
Epoch: 1810 loss_train: 2.3178 acc_train: 0.1481 loss_val: 2.3102 acc_val: 0.1510 time: 0.3866s
Epoch: 1811 loss_train: 2.3173 acc_train: 0.1525 loss_val: 2.3103 acc_val: 0.1516 time: 0.4170s
Epoch: 1812 loss_train: 2.3209 acc_train: 0.1412 loss_val: 2.3106 acc_val: 0.1521 time: 0.4214s
Epoch: 1813 loss_train: 2.3224 acc_train: 0.1472 loss_val: 2.3113 acc_val: 0.1516 time: 0.3973s
Epoch: 1814 loss_train: 2.3185 acc_train: 0.1491 loss_val: 2.3121 acc_val: 0.1531 time: 0.3979s
Epoch: 1815 loss_train: 2.3160 acc_train: 0.1400 loss_val: 2.3130 acc_val: 0.1516 time: 0.3992s
Epoch: 1816 loss_train: 2.3173 acc_train: 0.1422 loss_val: 2.3138 acc_val: 0.1505 time: 0.3817s
Epoch: 1817 loss_train: 2.3190 acc_train

Epoch: 1893 loss_train: 2.3233 acc_train: 0.1465 loss_val: 2.3128 acc_val: 0.1526 time: 0.4263s
Epoch: 1894 loss_train: 2.3180 acc_train: 0.1534 loss_val: 2.3128 acc_val: 0.1510 time: 0.3801s
Epoch: 1895 loss_train: 2.3198 acc_train: 0.1346 loss_val: 2.3127 acc_val: 0.1521 time: 0.4015s
Epoch: 1896 loss_train: 2.3201 acc_train: 0.1435 loss_val: 2.3125 acc_val: 0.1516 time: 0.3858s
Epoch: 1897 loss_train: 2.3221 acc_train: 0.1465 loss_val: 2.3121 acc_val: 0.1526 time: 0.3831s
Epoch: 1898 loss_train: 2.3185 acc_train: 0.1465 loss_val: 2.3119 acc_val: 0.1536 time: 0.3850s
Epoch: 1899 loss_train: 2.3222 acc_train: 0.1331 loss_val: 2.3117 acc_val: 0.1526 time: 0.3847s
Epoch: 1900 loss_train: 2.3270 acc_train: 0.1388 loss_val: 2.3115 acc_val: 0.1536 time: 0.4077s
Epoch: 1901 loss_train: 2.3145 acc_train: 0.1353 loss_val: 2.3114 acc_val: 0.1531 time: 0.4085s
Epoch: 1902 loss_train: 2.3242 acc_train: 0.1494 loss_val: 2.3115 acc_val: 0.1510 time: 0.3807s
Epoch: 1903 loss_train: 2.3157 acc_train

Epoch: 1979 loss_train: 2.3147 acc_train: 0.1440 loss_val: 2.3106 acc_val: 0.1510 time: 0.4250s
Epoch: 1980 loss_train: 2.3191 acc_train: 0.1368 loss_val: 2.3105 acc_val: 0.1521 time: 0.4002s
Epoch: 1981 loss_train: 2.3158 acc_train: 0.1381 loss_val: 2.3107 acc_val: 0.1516 time: 0.4218s
Epoch: 1982 loss_train: 2.3142 acc_train: 0.1447 loss_val: 2.3111 acc_val: 0.1531 time: 0.3751s
Epoch: 1983 loss_train: 2.3084 acc_train: 0.1463 loss_val: 2.3114 acc_val: 0.1536 time: 0.4172s
Epoch: 1984 loss_train: 2.3182 acc_train: 0.1374 loss_val: 2.3119 acc_val: 0.1536 time: 0.4236s
Epoch: 1985 loss_train: 2.3150 acc_train: 0.1620 loss_val: 2.3123 acc_val: 0.1526 time: 0.4163s
Epoch: 1986 loss_train: 2.3189 acc_train: 0.1482 loss_val: 2.3126 acc_val: 0.1546 time: 0.3904s
Epoch: 1987 loss_train: 2.3167 acc_train: 0.1437 loss_val: 2.3127 acc_val: 0.1541 time: 0.4235s
Epoch: 1988 loss_train: 2.3180 acc_train: 0.1404 loss_val: 2.3126 acc_val: 0.1526 time: 0.3945s
Epoch: 1989 loss_train: 2.3168 acc_train

In [15]:
# Testing
test_dict = {}
test_dict['features'] = features
test_dict['adj'] = adj
test_dict['labels'] = labels
test_dict['idx_test'] = idx_test
test(model, test_dict)

Test set results: loss= 2.5213 accuracy= 0.0725


In [16]:
model_name = 'model.pickle'
pickle.dump(model, open(result_path+model_name,'wb'))